In [2]:
import torch
import torch.nn as nn
import random
from datasets import load_dataset

# === Parameters ===
SEQ_LENGTH = 75
EMBED_DIM = 64
HIDDEN_DIM = 128
NUM_LAYERS = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === Load data to get vocabulary ===
print("Loading dataset and vocab...")
dataset = load_dataset('sander-wood/irishman', split='train')
texts = dataset['abc notation']
vocab = sorted(set(''.join(texts)))
char2idx = {ch: i for i, ch in enumerate(vocab)}
idx2char = {i: ch for ch, i in char2idx.items()}
VocabSize = len(vocab)

# === Define model ===
class SimpleRNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.embed = nn.Embedding(VocabSize, EMBED_DIM)
        self.lstm = nn.LSTM(EMBED_DIM, HIDDEN_DIM, NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(HIDDEN_DIM, VocabSize)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = out.reshape(-1, HIDDEN_DIM)
        logits = self.fc(out)
        return logits, hidden

# === Load model ===
model = SimpleRNN().to(DEVICE)
model.load_state_dict(torch.load("best_model.pth", map_location=DEVICE))
model.eval()
print("Model loaded successfully!")

# === Generate ABC tune ===
M_options = ["M:6/8", "M:4/4", "M:3/4"]
L_options = ["L:1/8", "L:1/16", "L:1/4"]
K_options = ["K:D", "K:G", "K:C"]

start = f"""X:1
T:Generated Tune
{random.choice(M_options)}
{random.choice(L_options)}
{random.choice(K_options)}
"""

seq = [char2idx.get(c, 0) for c in start]
hidden = None

for _ in range(800):  # generate 800 characters
    inp_seq = seq[-SEQ_LENGTH:]
    inp = torch.tensor(inp_seq).unsqueeze(0).to(DEVICE)
    logits, hidden = model(inp, hidden)
    probs = torch.softmax(logits[-1], dim=0)
    idx = torch.multinomial(probs, 1).item()
    seq.append(idx)

# === Decode and print ===
generated = ''.join(idx2char[i] for i in seq)
print("\n=== Generated ABC Notation ===\n")
print(generated)


Loading dataset and vocab...


C:\Users\geome\AppData\Local\Temp\ipykernel_1204\4220505250.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_locat

Model loaded successfully!

=== Generated ABC Notation ===

X:1
T:Generated Tune
M:6/8
L:1/8
K:G
 e2 | e2 f2 e4 | e2 c2 BABc | dBge dcBA | G2 G2 A2 G2 :| d2 e2 efga | gfed e2 ef | 
 gfga g2 g2 | e2 g2 G2 G2 | g2 gf g2 e2 | dcBA G2 D2 | G2 G2 G2 B2 | BcdB A2 A2 | 
 Bcde d2 B2 | c2 BA A2 A2 | B2 G2 BABG | G2B2 d2 G2 | BdBA G4 :: D2GG BAGG | B2G2 A4 | 
 BGGA B2B2 | [AA]2AB A2G2 | G2GG G2G2 :| (de/f/g)g g2 A4 | d2df a2(ba) | g4{g} f4 e2 | 
 dged dBG2 | f2ef{df} e2d2 | dcBA TB2>e2 | (dBg)B (ABA2) | GBG2 (B2G2) | e2a2g2 (de) | 
 d2B2 G4 || d2B2c2 e2d2>B2 | B2E2E2 c2B2g2 | e2>d'2 e'2c'2 A2A2 | E2>A2 G2>A2 B4 | 
 e2>e2 E2(3cBA e2e2 | c2>A2 c2B2 A2GA B2c2 | d2>e2 c2A2 A2G2 G2DE | G2AG E2>G2 G2A2 d2BG | 
 A2e2 A2d2 ^cede c2AG E2G2 | G2B2 G2>B2d2c2 Bcde d^cde | ^c2A2 ^G2A2 EAcd B2GE G4 g4 :: 
 G2B2 G2>^A2B2 G2G2G2 G2B2 AG2F F2>F2 | E2>A2 G2<E2 B4 A8 A4 B2c2 | B8- B4 B8 B4 :: 
[M:6/8] B4 | A8 B2A
